In [1]:
import torch
import torchaudio.functional as F

from denoiser.data.audio import Audio
from denoiser.data.augmentations import BackgroundNoise
from IPython.display import Audio as AudioPlayer

In [25]:
x = Audio('/data/denoising/speech/daps/clean/f10_script1_clean.wav')
bnoise = BackgroundNoise('/data/denoising/noise/records/DNC/index.json', min_snr=-5.0, max_snr=25.0, p=1.0)

In [ ]:
AudioPlayer(x.random_excerpt(1.7).waveform.numpy(), rate=x.sample_rate)

In [ ]:
excerpt = x.random_excerpt(1)
AudioPlayer(excerpt.waveform.numpy(), rate=excerpt.sample_rate)

In [ ]:
augmented, noise = bnoise(excerpt)
AudioPlayer(augmented.waveform.numpy(), rate=augmented.sample_rate)

In [ ]:
AudioPlayer(noise.waveform.numpy(), rate=noise.sample_rate)